In [116]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
relation_feat_lst = []
relation_feat_syn = None
relation_feat_ant = None
relation_feat_hype = None
relation_feat_hypo = None
relation_feat_same_hype = None
def PrintRelationLst():
    print('relation-syn:',relation_feat_syn)
    print('relation-ant:',relation_feat_ant)
    print('relation-hype:',relation_feat_hype)
    print('relation-hypo:',relation_feat_hypo)
    print('relation-same_hype:',relation_feat_same_hype)

## Synonymy & Antonymy

In [117]:
# WORD1 = 'good'
# WORD2 = 'beneficial'
# WORD1 = 'wet'
# WORD2 = 'dry'
WORD1 = 'wet'
WORD2 = 'dry'
syn_flg = False
WORD1_syn_lst = wn.synsets(WORD1)
WORD2_syn_lst =wn.synsets(WORD2)
# synonymy
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        if word1_syn == word2_syn:
            relation_feat_syn = 1
    
# antonymy
syn_flg = True
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        if word1_syn.lemmas()[0].antonyms():
#             print('word1:',word1_syn.lemmas()[0])
#             print('word2:',word2_syn.lemmas()[0])
#             print('============================')
            if word1_syn.lemmas()[0].antonyms()[0] == word2_syn.lemmas()[0]:
                relation_feat_ant = 1
if not relation_feat_syn:
    relation_feat_syn = 0
if not relation_feat_ant:
    relation_feat_ant = 0
PrintRelationLst()

relation-syn: 0
relation-ant: 1
relation-hype: None
relation-hypo: None
relation-same_hype: None


## Hypernymy & Hpyponymy & Same-Hypernym

In [119]:
# here due to its external knowledge definition, we will simple consider Hypernymy and Hyponymy as the same value
# =========== def ===========
# takes value 1 - n/8 if one word is a (direct or indirect) hypernym of the other wors in WordNet. 
# Where n is the number of edges between the two words in hierarchies, and 0 otherwise.
# Eg. [dog, canid] = 0.875
# Eg. [wolf, canid] = 0.875
# Eg. [dog, carnivore] = 0.75
# Eg. [canid, dog] = 0
# ======== End def ==========

## first find their most similarity def
print('word1 = ',WORD1)
print('word2 = ',WORD2)
print('================')
similarity = -np.float('inf')
for word1_i, word1_syn in enumerate(WORD1_syn_lst):
    for word2_i, word2_syn in enumerate(WORD2_syn_lst):
        sim_val = wn.wup_similarity(word1_syn, word2_syn)
        if sim_val: # if they exist similaroty value
            if sim_val > similarity:
                best_val = sim_val
                best_syn1 = word1_syn
                best_syn2 = word2_syn
                similarity = sim_val
# print(best_syn1)
# print(best_syn2)
        
## base on the def, to figure out their hyper-path
best_syn1_path = best_syn1.hypernym_paths()
for idx, path in enumerate(best_syn1_path):
    for i, hyper in enumerate(path):
        if hyper == best_syn2:
            edgN = len(path)-i-1
            relation_feat_hype = 1-edgN/8
            break
    if relation_feat_hype:
        break
    
best_syn2_path = best_syn2.hypernym_paths()
for idx, path in enumerate(best_syn2_path):
    for i, hyper in enumerate(path):
        if hyper == best_syn1:
            edgN = len(path)-i-1
            relation_feat_hypo = 1-edgN/8
            break
    if relation_feat_hypo:
        break

if relation_feat_hype:
    relation_feat_hypo = relation_feat_hype
elif relation_feat_hypo:
    relation_feat_hype = relation_feat_hypo
else:
    relation_feat_hype = 0
    relation_feat_hypo = 0
    
## Same-Hypernym
if relation_feat_syn == 0:
    # to see if the two words have the same hypernym
    for syn1_idx, syn1_path in enumerate(best_syn1_path):
        for syn2_idx, syn2_path in enumerate(best_syn2_path):
            for syn1_hyper in syn1_path:
                for syn2_hyper in syn2_path:
                    if syn1_hyper == syn2_hyper:
#                         print(syn1_hyper)
#                         print(syn2_hyper)
                        relation_feat_same_hype = 1
if not relation_feat_same_hype:
    relation_feat_same_hype = 0
# PrintRelationLst() 

word1 =  wet
word2 =  dry


In [120]:
PrintRelationLst()

relation-syn: 0
relation-ant: 1
relation-hype: 0
relation-hypo: 0
relation-same_hype: 0


In [ ]:
## External knowledge
λ1(rij ), where λ
is a hyper-parameter tuned on the development set and 1 is the indication function.
1(rij ) = 
1 if rij is not zero vector ;
0 if rij is zero vector .